### Important:
This file must be run using **Anaconda** with the following packages:
- numpy
- pandas
- openpyxl
- osgeo.gdal

**And you must set these to match keys in your hsi_config.json file:**

In [1]:
# MUST SET TO MATCH KEYS IN HSI_CONFIG
HOG_ISLAND_2019_0 = "uas_hsi_small"
HOG_ISLAND_2019_1 = "uas_hsi_large"

In [2]:
from modules.environment_manager import get_notebook_path, get_persistent_config_data

NOTEBOOK_NAME = "geo_image_construction.ipynb"

NOTEBOOK_PATH = get_notebook_path(NOTEBOOK_NAME)
THREADS, MEMORY, HSI_CONFIG, PROJECT_FOLDER, OUTPUT_FOLDER = get_persistent_config_data(NOTEBOOK_PATH)

In [3]:
import math
import pandas as pd

DATA_FOLDER = PROJECT_FOLDER / "data"
PROCESSED_FOLDER = DATA_FOLDER / "processed"

In [4]:
def get_vp_geo_df():
    df = pd.DataFrame()
    for geo_fp in (DATA_FOLDER / "vp_geo").rglob("*"):
        if geo_fp.suffix == ".csv":
            geo_df = pd.read_csv(geo_fp)
            geo_df.columns = [col.strip().lower() for col in geo_df.columns]
            wavelengths, vp_name = geo_df.columns
            if not wavelengths == "wavelength":
                print("fail wavelength name check", geo_fp)
            if not vp_name.startswith("vp"):
                print("fail vp name check", geo_fp)
            if not math.isclose(geo_df["wavelength"][201], 899.41803):
                print("fail wavelength isclose check", geo_fp)
            if not geo_df[vp_name][:202].round(5).nunique() == 1:
                print("fail all same check", geo_fp)
            vp_num = int(vp_name[2:])
            new_vp_name = "vp_" + str(vp_num).zfill(3)
            colname = geo_fp.parent.name
            df.loc[new_vp_name, colname] = geo_df[vp_name][0]
        elif geo_fp.is_dir():
            df[geo_fp.name] = None

    assert "solar_azimuth" in df.columns
    assert "sensor_azimuth" in df.columns

    df["relative_azimuth"] = df["solar_azimuth"] - df["sensor_azimuth"]
    df["relative_azimuth"] = (df["relative_azimuth"].mod(360) + 360).mod(360)  # adjust to be between 0 and 360
    df = df.sort_index()
    df = df.sort_index(axis=1)
    return df

vp_geo_df = get_vp_geo_df()
vp_geo_df.to_csv(PROCESSED_FOLDER / "vp_geo.csv")
vp_geo_df

,relative_azimuth,sensor_azimuth,sensor_zenith,solar_azimuth,solar_zenith
vp_003,105.634543,4.895457,13.037619,110.53,35.53
vp_004,105.644973,4.885027,10.211479,110.53,35.53
vp_005,105.612238,4.907762,12.109447,110.52,35.53
vp_006,105.689935,4.830065,9.422757,110.52,35.53
vp_008,284.403807,186.116193,1.278386,110.52,35.53
vp_009,285.495573,185.034427,10.772873,110.53,35.52
vp_010,106.835811,3.674189,0.897489,110.51,35.53
vp_011,102.853457,7.656543,4.640381,110.51,35.54
vp_013,284.313169,185.646831,9.702342,109.96,35.95
vp_014,284.200041,185.759959,6.265679,109.96,35.95


In [5]:
def get_vp_loc_df():
    df = pd.read_excel(DATA_FOLDER / "vp_loc" / "hog_2019_marsh_utm_average.xlsx", index_col="Sample ID")
    df = df[["Northing (m)", "Easting (m)"]]
    df.columns = ["northing", "easting"]
    df.index = df.index.str.strip().str.lower()
    df = df[df.index.str.startswith("vp")]
    new_index = []
    for sample in df.index:
        parts = sample.split("_")
        vp_part = "vp_" + parts[1].zfill(3)
        if len(parts) > 2:
            vp_part += "_" + parts[2].zfill(3)
        else:
            vp_part += "_001"
        new_index.append(vp_part)
    df.index = new_index  # type: ignore
    df["first_num"] = df.index.str.split("_").str[1].astype(int)  # type: ignore
    df["second_num"] = df.index.str.split("_").str[2].astype(int)  # type: ignore
    df = df.sort_values(by=["first_num", "second_num"]).drop(columns=["first_num", "second_num"])
    df.sort_index(axis=1)
    assert df.isna().any().any() == False
    return df


vp_loc_df = get_vp_loc_df()
vp_loc_df.to_csv(PROCESSED_FOLDER / "vp_loc.csv")
vp_loc_df

,northing,easting
vp_001_001,4138686.094,437051.979
vp_001_002,4138685.694,437052.351
vp_001_003,4138685.345,437051.968
vp_001_004,4138685.706,437051.740
vp_002_001,4138681.245,437048.546
...,...,...
vp_107_004,4138056.755,436868.246
vp_108_001,4138044.313,436861.612
vp_108_002,4138044.243,436861.118
vp_108_003,4138044.703,436861.106


In [6]:
import numpy as np
from osgeo import gdal, osr
gdal.UseExceptions()

In [7]:

def envi_to_utm_coordinates_img(envi_image_path, utm_zone=18):
    # Open the ENVI hyperspectral image
    dataset = gdal.Open(envi_image_path)
    
    if not dataset:
        raise FileNotFoundError("Unable to open the ENVI hyperspectral image.")
    
    # Get the image dimensions
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    
    # Get the geotransform and projection
    geotransform = dataset.GetGeoTransform()
    projection = dataset.GetProjection()
    
    # Create a spatial reference object for the original projection
    source_srs = osr.SpatialReference()
    source_srs.ImportFromWkt(projection)
    
    # Create a spatial reference object for UTM zone 18N
    target_srs = osr.SpatialReference()
    target_srs.SetUTM(utm_zone, True)  # True for northern hemisphere
    
    # Create a coordinate transformation
    transform = osr.CoordinateTransformation(source_srs, target_srs)
    
    # Create meshgrid for pixel coordinates
    x = np.arange(width)
    y = np.arange(height)
    xx, yy = np.meshgrid(x, y)
    
    # Convert pixel coordinates to geospatial coordinates
    x_geo = geotransform[0] + xx * geotransform[1] + yy * geotransform[2]
    y_geo = geotransform[3] + xx * geotransform[4] + yy * geotransform[5]
    
    # Flatten the arrays for vectorized transformation
    x_geo_flat = x_geo.flatten()
    y_geo_flat = y_geo.flatten()
    
    # Transform coordinates
    utm_coords_flat = np.array(transform.TransformPoints(np.column_stack((x_geo_flat, y_geo_flat))))
    
    # Reshape to original image dimensions
    utm_coords = utm_coords_flat[:, [1, 0]].reshape(height, width, 2)
    
    return utm_coords

utm_coordinates_0 = envi_to_utm_coordinates_img(envi_image_path=HSI_CONFIG[HOG_ISLAND_2019_0]["img"])
utm_coordinates_1 = envi_to_utm_coordinates_img(envi_image_path=HSI_CONFIG[HOG_ISLAND_2019_1]["img"])
np.save(PROCESSED_FOLDER / "utm_hog_island_2019_0.npy", arr=utm_coordinates_0)
np.save(PROCESSED_FOLDER / "utm_hog_island_2019_1.npy", arr=utm_coordinates_1)
print(utm_coordinates_0.shape)
print(utm_coordinates_1.shape)

(3705, 3217, 2)
(5321, 5417, 2)


In [8]:
def map_to_utm_coordinates_img_idx_pix(northings, eastings, *utm_coords):
    if len(northings) != len(eastings):
        raise ValueError("The length of northings and eastings must be equal.")
    
    pixel_coords = []
    img_indices = []
    for n, e in zip(northings, eastings):
        min_distance = float('inf')
        min_idx = None
        min_img_idx = None
        
        # Iterate over each UTM image coordinates
        for img_idx, utm in enumerate(utm_coords):
            if utm.ndim != 3 or utm.shape[2] != 2:
                raise ValueError(f"UTM coordinates array at index {img_idx} must have shape (height, width, 2).")
    
            # Calculate the distances between the point and all pixels in the current image
            distances = np.sqrt((utm[:, :, 0] - n)**2 + (utm[:, :, 1] - e)**2)
            current_min_idx = np.unravel_index(np.argmin(distances), distances.shape)
            current_min_distance = distances[current_min_idx]
            
            # Check if the current image has a closer pixel
            if current_min_distance < min_distance:
                min_distance = current_min_distance
                min_idx = current_min_idx
                min_img_idx = img_idx
        
        # Append the closest pixel coordinates and the corresponding image index
        pixel_coords.append(min_idx)
        img_indices.append(min_img_idx)
    
    return img_indices, pixel_coords

northings = vp_loc_df['northing'].values
eastings = vp_loc_df['easting'].values
img_idxs, pix_yx = map_to_utm_coordinates_img_idx_pix(northings, eastings, utm_coordinates_0, utm_coordinates_1)
vp_pix_df = pd.DataFrame({
    'vp_names': list(vp_loc_df.index),
    'img_idx': img_idxs,
    'pix_y': [pix[0] for pix in pix_yx],
    'pix_x': [pix[1] for pix in pix_yx]
})
vp_pix_df.set_index('vp_names', inplace=True)
vp_pix_df.to_csv(PROCESSED_FOLDER / "vp_pix.csv")
vp_pix_df

,img_idx,pix_y,pix_x
vp_names,,,
vp_001_001,0,1721,1857
vp_001_002,0,1728,1863
vp_001_003,0,1734,1857
vp_001_004,0,1728,1852
vp_002_001,0,1808,1796
...,...,...,...
vp_107_004,1,3060,3919
vp_108_001,1,3277,3805
vp_108_002,1,3278,3797
